In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_114190/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [14]:
import pandas as pd
pandas_seed = 42
import numpy as np
import timeit

import random
import string

import os

import torch
from torch import nn, save
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
torch.manual_seed(42)

from torchvision.transforms import ToTensor

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [15]:
DNA_ONE_HOT = {}

# E stand for end, which is the head or the tail of the DNA strand
ct = 0
for pos1 in ["E","A","G","C","T"]:
    for pos2 in ["E","A","G","C","T"]:
            comb = pos1+pos2
            if pos1 == "E" and pos2 =="E":
                continue
            elif comb in DNA_ONE_HOT.keys():
                continue
            else:
                DNA_ONE_HOT[comb] = [ int(x) for x in list(f'{ct:05b}') ]  
                ct += 1
print("E stand for end, which is the head or the tail of the DNA strand")
print(len(DNA_ONE_HOT))
for idx, ele in DNA_ONE_HOT.items():
    print(idx,ele)



E stand for end, which is the head or the tail of the DNA strand
24
EA [0, 0, 0, 0, 0]
EG [0, 0, 0, 0, 1]
EC [0, 0, 0, 1, 0]
ET [0, 0, 0, 1, 1]
AE [0, 0, 1, 0, 0]
AA [0, 0, 1, 0, 1]
AG [0, 0, 1, 1, 0]
AC [0, 0, 1, 1, 1]
AT [0, 1, 0, 0, 0]
GE [0, 1, 0, 0, 1]
GA [0, 1, 0, 1, 0]
GG [0, 1, 0, 1, 1]
GC [0, 1, 1, 0, 0]
GT [0, 1, 1, 0, 1]
CE [0, 1, 1, 1, 0]
CA [0, 1, 1, 1, 1]
CG [1, 0, 0, 0, 0]
CC [1, 0, 0, 0, 1]
CT [1, 0, 0, 1, 0]
TE [1, 0, 0, 1, 1]
TA [1, 0, 1, 0, 0]
TG [1, 0, 1, 0, 1]
TC [1, 0, 1, 1, 0]
TT [1, 0, 1, 1, 1]


In [16]:
result_log = []


PADDING = True 
    
MAX_LEN = 60
N_FEATURES = 5
N_percentage = 0.25 # 1 - 1M

DATA_USED_N = 1000000 # max is 1000000
# DATA_USED_N = 60 # max is 1000000

In [17]:

data_org = pd.read_csv("./data/1mill_dataset.csv", names=["strand","energy"],dtype={0: str, 1: float})
print("len(data_org)=",len(data_org))
data_org.head()

len(data_org)= 1000000


,strand,energy
0,GAATCTTCGCACTCTAGCTGACCGCCTTCAGTAGTACGAATCTGGA...,-5.329171
1,AAGAAGCGGGAAGCATATCTTTATTCAGTTCCTAAT,-1.687614
2,CCTAATCGGGAATTGTTCCTCTTCCATTTGTAATGGTTATAAGAGG...,-7.224524
3,CACCGGGGCATTTATCCCGGGCTCGAAGGAAGTCTTGG,-7.222121
4,CTCGGGAAGCAAAACCCCTAG,-1.601845


In [18]:
shuffled_df = data_org.sample(frac = 1, random_state=pandas_seed)
shuffled_df.reset_index(inplace=True, drop=True)

# # Clip data, too large can not train in local

shuffled_df = shuffled_df.loc[:DATA_USED_N-1]


train_pct = 0.7
val_pct = 0.15
test_pct = 1 - train_pct - val_pct

# # shuffled_df.head()
all_n = len(shuffled_df)
split1 = int(train_pct * all_n) 
split2 = int((train_pct + val_pct) * all_n) 

train_df_org = shuffled_df.loc[0:split1-1].copy()
val_df = shuffled_df.loc[split1:split2-1].copy()
test_df = shuffled_df.loc[split2:,:].copy()

train_df_reverse = train_df_org.copy()
train_df_reverse["strand"] = train_df_reverse.loc[:,'strand'].apply(lambda x: x[::-1])

train_df_all = pd.concat([train_df_org, train_df_reverse])

print(f"total {all_n}, train idx 0 - {split1-1}, val idx{split1} - {split2-1}, test idx {split2} - {all_n-1}")
print(f"\n\nlen(train_df) : {len(train_df_org) }, \ntrain_df_all(with reverse) {len(train_df_all)} \nlen(val_df) : {len(val_df)},\nlen(test_df) : {len(test_df)}")


train_df = train_df_all

train_val_df = shuffled_df.loc[:split2-1]
train_val_mean = train_val_df["energy"].mean()
train_val_stdev = train_val_df["energy"].std()

total 1000000, train idx 0 - 699999, val idx700000 - 849999, test idx 850000 - 999999


len(train_df) : 700000, 
train_df_all(with reverse) 1400000 
len(val_df) : 150000,
len(test_df) : 150000


In [19]:
def convert_dna_to_threegrain(str1):
    str1 = "E" + str1 + "E"
    threegrain = []
    length = len(str1)
    for idx in range(0,length-1,1):
        threegrain.append(str1[idx:idx+2])
        
    return threegrain

print(convert_dna_to_threegrain("AGCTAT"))


def x_transform(dna_str, pad=PADDING):
    n_pad = MAX_LEN - len(dna_str)
    threegrain_list =  convert_dna_to_threegrain(dna_str)
    target = np.array(threegrain_list)
    onehot = np.array([DNA_ONE_HOT[tri_letter] for tri_letter in threegrain_list])
#     one_hot_paded = np.pad(onehot,((0,n_pad),(0,0)), mode='constant')
#     print(onehot.shape, fea.shape)
    if pad == True:
        return np.pad(onehot,((0,n_pad),(0,0)), mode='constant')
    else:
        return onehot
    
    
def y_transform(energy_float):
    y_trans = (energy_float - train_val_mean) / train_val_stdev
    return y_trans   

def y_transform_reverce(y_array):
    y_arr = y_array.cpu().numpy()
    y_pred = y_arr * train_val_stdev + train_val_mean
    return y_pred


['EA', 'AG', 'GC', 'CT', 'TA', 'AT', 'TE']


In [20]:
# x_transform("CGTA")

In [21]:
class TransformDataset(Dataset):
    def __init__(self, df, x_transform, y_transform):
        self.x_transform = x_transform
        self.y_transform = y_transform
        self.X = df["strand"].values
        self.y = df["energy"].values

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx): 
        x = self.x_transform(self.X[idx])
        y = self.y_transform(self.y[idx])

        return torch.tensor(x), torch.tensor(y)

# train_dataset = GetData(train_df, x_transform, y_transform)
# X,y = train_dataset[1]
# print(X.shape)
# print(y)

train_dataset = TransformDataset(train_df, x_transform, y_transform)
val_dataset = TransformDataset(val_df, x_transform, y_transform)
test_dataset = TransformDataset(test_df, x_transform, y_transform)


# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# X,y = next(iter(train_loader))
# print("Features shape:", X.shape)
# print("Target shape:", y.shape)

In [22]:
class LSTM_DNA(nn.Module):
    def __init__(self, input_dim_n, hidden_units, num_layers,is_bi_dir):
        super(LSTM_DNA, self).__init__()
        self.input_dim = input_dim_n
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.direction_n = (1 + int(is_bi_dir))
        
        self.lstm = nn.LSTM(
            input_size = input_dim_n,
            hidden_size = hidden_units,
            batch_first = True,
            num_layers = self.num_layers,
            bidirectional = is_bi_dir
        )
        
        self.linear1 = nn.Linear(in_features = self.hidden_units, out_features=45)
        self.linear2 = nn.Linear(in_features = 45, out_features=1)
        
    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers * self.direction_n, batch_size, self.hidden_units).to(device).requires_grad_()
        c0 = torch.zeros(self.num_layers * self.direction_n, batch_size, self.hidden_units).to(device).requires_grad_()
        _, (hn, _) = self.lstm(x, (h0, c0))
#         out1 = self.linear1(hn[0]).flatten()
        out1 = self.linear1(hn[0])
#         print(out1.shape)
        out2 = self.linear2(out1).flatten()
        
        return out2

In [23]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0.0
    model.train()
    
    batch_i = 1
    model = model.float()
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        output = model(X.float())
        loss = loss_function(output, y.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        batch_i += 1
#         if batch_i % 200 == 0:
#             print(f"batch {batch_i}, loss = {loss.item()}")
    
    avg_loss = total_loss / num_batches 
#     print(f"Train loss: {avg_loss}")
    return avg_loss
    
def eval_model(data_loader, model, loss_function):
    num_batches = len(data_loader)
    total_loss = 0.0
    
    model.eval()
    
    model = model.float()
    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(device)
            y = y.to(device)
            output = model(X.float())
            total_loss += loss_function(output, y.float()).item()
    
    avg_loss = total_loss / num_batches
#     print(f"   Evalulation loss: {avg_loss}")
    return avg_loss
    

def predict(data_loader, model):

    output = torch.tensor([])
    output = output.to(device)
    model.eval()
    model = model.float()
    with torch.no_grad():
        for X, _ in data_loader:
            X = X.to(device)
            y_out = model(X.float())
            output = torch.cat((output, y_out.float()), 0)
    
    energy = y_transform_reverce(output)
    
    return energy

def get_avg_mse(y, y_pred):
    mse = ((y - y_pred)**2).mean()
    print(f"      current mse: {mse}")
    return mse


In [ ]:
hyper_paras_list = [
    [150, 1500, 0.01, 90, 2, True],
    [150, 1500, 0.01, 120, 2, True],
    [150, 1500, 0.01, 140, 2, True]
]

result_df = pd.DataFrame(columns = ["EPOCH", "BATCH_SIZE", "LR", "N_HIDDEN_UNITS", "NUM_LAYERS","data_used_n","min_val_loss_id", "min_val_loss", "test_mse", "train_loss_list", "val_loss_list"])


for paras in hyper_paras_list:
    EPOCH,BATCH_SIZE,LR,N_HIDDEN_UNITS,NUM_LAYERS, BI_DIR = paras
    
    model  = LSTM_DNA(input_dim_n = N_FEATURES, hidden_units=N_HIDDEN_UNITS, num_layers=NUM_LAYERS, is_bi_dir=BI_DIR)
    model = model.float()
    model.to(device)
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LR)
    
#     train_dataset, val_dataset, test_dataset = train_dataset.to(device), val_dataset.to(device), test_dataset.to(device)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model_list = []
    train_loss_list = []
    val_loss_list = []

    print(f"EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, is_bi_dir = {EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, BI_DIR}")
    
    min_val_loss = 1e8
    lowest_epoch = 0
    PATH = f"EPOCH{EPOCH}BATCH_SIZE{BATCH_SIZE}LR{LR}N_HIDDEN_UNITS{N_HIDDEN_UNITS}NUM_LAYERS{NUM_LAYERS}BI_DIR{BI_DIR}.pt"
    print(PATH)
    
    patient = 0
    
    for ix_epoch in range(EPOCH):
#         print(f"Epoch {ix_epoch} ")
        start = timeit.default_timer()
        train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
        val_loss = eval_model(val_loader, model, loss_function)
        
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        
        if val_loss < min_val_loss:
                if os.path.exists(PATH):
                    os.remove(PATH)
                torch.save(model.state_dict(), PATH)
                min_val_loss = val_loss
                lowest_epoch = ix_epoch
                patient = 0
                
        elif val_loss > min_val_loss * 2:
            break
            
        else:
            patient += 1
            
        
        stop = timeit.default_timer()
        print(f'Epoch {ix_epoch} finished val_loss {val_loss}, Time Elapse: ', stop - start, "patient=", patient) 
        
        if patient >= 6:
            break
        
    min_val_loss_id = np.argmin(val_loss_list)
    min_val_loss = val_loss_list[min_val_loss_id]
    
    model = LSTM_DNA(input_dim_n = N_FEATURES, hidden_units=N_HIDDEN_UNITS, num_layers=NUM_LAYERS, is_bi_dir=BI_DIR)
    model.load_state_dict(torch.load(PATH))
    model.to(device)
    model.eval()
    
    energy_predict = predict(test_loader, model)
    test_mse = get_avg_mse(test_df["energy"].values,energy_predict)
    
    print(f"Total Epoch {EPOCH}\n min_val_id {min_val_loss_id}, min_val_loss {min_val_loss}, test_mse {test_mse}")

    model_result = [EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS,BI_DIR, DATA_USED_N,min_val_loss_id, min_val_loss, test_mse, train_loss_list, val_loss_list]

    current_df = pd.DataFrame([model_result], columns = ["EPOCH", "BATCH_SIZE", "LR", "N_HIDDEN_UNITS", "NUM_LAYERS","BI_DIR","data_used_n" ,"min_val_loss_id", "min_val_loss", "test_mse", "train_loss_list", "val_loss_list"])

    result_df = pd.concat([result_df,current_df])
    letters = string.ascii_lowercase
    suffix = ''.join(random.choice(letters) for i in range(10))
    print(suffix)
    result_df["note"] = "multi layers for output neural network with 2 grains feature engineering"
    result_df.to_csv(f"2022-05-01_1_{suffix}.csv")

EPOCH, BATCH_SIZE, LR, N_HIDDEN_UNITS, NUM_LAYERS, is_bi_dir = (150, 1500, 0.01, 90, 2, True)
EPOCH150BATCH_SIZE1500LR0.01N_HIDDEN_UNITS90NUM_LAYERS2BI_DIRTrue.pt
Epoch 0 finished val_loss 0.41496407449245454, Time Elapse:  247.05748301299172 patient= 0
Epoch 1 finished val_loss 0.36412972897291185, Time Elapse:  247.86586745700333 patient= 0
Epoch 2 finished val_loss 0.3284513261914253, Time Elapse:  247.4604302099906 patient= 0
Epoch 3 finished val_loss 0.31161707639694214, Time Elapse:  247.6984825609834 patient= 0
Epoch 4 finished val_loss 0.2943620117008686, Time Elapse:  246.82571674499195 patient= 0
Epoch 5 finished val_loss 0.28727971464395524, Time Elapse:  247.8459660759836 patient= 0
Epoch 6 finished val_loss 0.2767387245595455, Time Elapse:  248.02467365699704 patient= 0
Epoch 7 finished val_loss 0.2713864703476429, Time Elapse:  247.95520609398955 patient= 0
Epoch 8 finished val_loss 0.2682210622727871, Time Elapse:  248.16135141800623 patient= 0
Epoch 9 finished val_loss 

Epoch 26 finished val_loss 0.2139306089282036, Time Elapse:  300.275279649999 patient= 3
Epoch 27 finished val_loss 0.2143271568417549, Time Elapse:  298.2768166780006 patient= 4
Epoch 28 finished val_loss 0.21234393671154975, Time Elapse:  301.38425462701707 patient= 0
Epoch 29 finished val_loss 0.2131918476521969, Time Elapse:  300.769796787994 patient= 1
Epoch 30 finished val_loss 0.2144676199555397, Time Elapse:  299.14139427701593 patient= 2
Epoch 31 finished val_loss 0.21339085429906846, Time Elapse:  299.54804418899585 patient= 3
Epoch 32 finished val_loss 0.21703603163361548, Time Elapse:  299.2857346330129 patient= 4
Epoch 33 finished val_loss 0.20990126475691795, Time Elapse:  298.97640593600227 patient= 0
Epoch 34 finished val_loss 0.21767018377780914, Time Elapse:  298.7326886130031 patient= 1
Epoch 35 finished val_loss 0.21504899755120277, Time Elapse:  299.865395522007 patient= 2
Epoch 36 finished val_loss 0.21057791501283646, Time Elapse:  298.79788302700035 patient= 3
E